In [1]:
import os
import json
import random
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
INPUT_JSONL = '/home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/splits/h5_tienkung_xsens_text_train_all.jsonl'
OUTPUT_DIR  = '/home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/splits'
SFT_JSONL   = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_text_sft.jsonl')
RL_JSONL    = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_text_rl.jsonl')
SPLIT_RATIO = 0.2   # SFT 占 20%，RL 占 80%
SEED = 42
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def group_by_trajectory(data):
    """
    将数据按完整 id（即轨迹）聚合。
    返回: {action_type: {trajectory_id: [items...] } }
    """
    grouped = defaultdict(lambda: defaultdict(list))
    for item in data:
        tid = item.get('id', '')
        act = extract_action_type(tid)
        if act:
            grouped[act][tid].append(item)
    return grouped

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def main():
    random.seed(SEED)
    ensure_dir(OUTPUT_DIR)

    # 1) 加载数据
    data = load_jsonl(INPUT_JSONL)

    # 2) 按类别 & 轨迹分组
    grouped = group_by_trajectory(data)
    print(f"✓ 共发现 {len(grouped)} 个类别（基于 extract_action_type）\n")

    # 3) 分层划分 SFT / RL
    sft_data, rl_data = [], []

    category_stats = []  # 用于打印统计
    for act, traj_map in grouped.items():
        traj_ids = list(traj_map.keys())
        random.shuffle(traj_ids)

        # 计算 SFT 轨迹数量
        sft_count = int(len(traj_ids) * SPLIT_RATIO)
        sft_traj = traj_ids[:sft_count]
        rl_traj = traj_ids[sft_count:]

        # 收集数据
        sft_items = [item for tid in sft_traj for item in traj_map[tid]]
        rl_items = [item for tid in rl_traj for item in traj_map[tid]]

        sft_data.extend(sft_items)
        rl_data.extend(rl_items)

        # 记录统计
        category_stats.append({
            '类别': act,
            'SFT_轨迹数': len(sft_traj),
            'RL_轨迹数': len(rl_traj),
            'SFT_样本数': len(sft_items),
            'RL_样本数': len(rl_items),
        })

    # 4) 写出文件
    dump_jsonl(sft_data, SFT_JSONL)
    dump_jsonl(rl_data, RL_JSONL)

    # 5) 打印统计信息
    print("\n====== 数据集划分完成 ======")
    print(f"总类别数: {len(category_stats)}")
    print(f"SFT : RL = {SPLIT_RATIO:.0%} : {1-SPLIT_RATIO:.0%}\n")

    print("类别分布统计:")
    print("------------------------------------------------------------")
    print(f"{'类别':25s} | {'SFT轨迹':7s} {'RL轨迹':7s} {'SFT样本':7s} {'RL样本':7s}")
    print("------------------------------------------------------------")
    for stat in category_stats:
        print(f"{stat['类别']:25s} | "
              f"{stat['SFT_轨迹数']:7d} {stat['RL_轨迹数']:7d} "
              f"{stat['SFT_样本数']:7d} {stat['RL_样本数']:7d}")
    print("------------------------------------------------------------")
    print(f"SFT 总样本: {len(sft_data)}   RL 总样本: {len(rl_data)}")
    print(f"👉 输出文件已生成:\n   - {SFT_JSONL}\n   - {RL_JSONL}\n")

if __name__ == '__main__':
    main()


✓ 成功加载 4687 条数据
✓ 共发现 19 个类别（基于 extract_action_type）


====== 数据集划分完成 ======
总类别数: 19
SFT : RL = 20% : 80%

类别分布统计:
------------------------------------------------------------
类别                        | SFT轨迹   RL轨迹    SFT样本   RL样本   
------------------------------------------------------------
battery_insertion_with_pullout |      11      46      66     262
cylinder_pick_box_place_close |      11      47      60     269
pick_shelf_insert_machine_press_switch_place_plate |       7      30      40     178
plate_push                |       8      35      24     111
place_button              |       9      39      51     185
plug_extract_from         |       8      32      51     187
place_button_then_press   |      11      44      68     253
plug_pullout              |       7      29      43     184
seal_stamping             |       9      36      57     233
nut_place                 |      10      42      61     278
plug_insertion_v2         |       9      37      48     239
plug_ins